# Ragnarok Mobile Review Scraping

In [ ]:
# Will be using this to get information on each app based on appid
# Information can be found here https://github.com/JoMingyu/google-play-scraper

#pip install google_play_scraper

In [ ]:
# Will help me scroll down a page to get all the possible html

#pip install selenium

In [1]:
import requests
import datetime
from datetime import datetime
import time
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver

==================================================================================================================

# Getting general information about the game

In [ ]:
#gametoreview = "https://play.google.com/store/apps/details?id=com.gravity.romNAg"

In [2]:
# This will do most of the work for us.

from google_play_scraper import app

# Doing a test run with a well-known app.
result = app(
    'com.nianticlabs.pokemongo',
    lang='en',
    country='us'
)

In [3]:
# Checking what I get in the dictionary.

result

{'title': 'Pokémon GO',
 'description': 'Join Trainers across the globe who are discovering Pokémon as they explore the world around them. Pokémon GO is the global gaming sensation that has been downloaded over 1 billion times and named “Best Mobile Game” by the Game Developers Choice Awards and “Best App of the Year” by TechCrunch.\r\n_______________\r\n\r\nUNCOVER THE WORLD OF POKÉMON: Explore and discover Pokémon wherever you are!\r\n \r\nCATCH more Pokémon to complete your Pokédex!\r\n \r\nJOURNEY alongside your Buddy Pokémon to help make your Pokémon stronger and earn rewards!\r\n\r\nCOMPETE in epic Gym battles and...\r\n\r\nTEAM UP with other Trainers to catch powerful Pokémon during Raid Battles!\r\n \r\nIt’s time to get moving—your real-life adventures await! Let’s GO!\r\n_______________\r\n\r\nNotes: \r\n\r\n- This app is free-to-play and offers in-game purchases. It is optimized for smartphones, not tablets.\r\n- Compatible with Android devices that have 2GB RAM or more and h

In [4]:
# Checking which keys I want to keep to use as features.

result.keys()

dict_keys(['title', 'description', 'descriptionHTML', 'summary', 'summaryHTML', 'installs', 'minInstalls', 'score', 'ratings', 'reviews', 'histogram', 'price', 'free', 'currency', 'sale', 'saleTime', 'originalPrice', 'saleText', 'offersIAP', 'inAppProductPrice', 'size', 'androidVersion', 'androidVersionText', 'developer', 'developerId', 'developerEmail', 'developerWebsite', 'developerAddress', 'privacyPolicy', 'developerInternalID', 'genre', 'genreId', 'icon', 'headerImage', 'screenshots', 'video', 'videoImage', 'contentRating', 'contentRatingDescription', 'adSupported', 'containsAds', 'released', 'updated', 'version', 'recentChanges', 'recentChangesHTML', 'comments', 'appId', 'url'])

In [5]:
featurekeys = ['title', 'installs', 'minInstalls', 'score', 'ratings', 'reviews', 'histogram', 'price', 'free', 'offersIAP', 'size', 'genre', 'genreId', 'released', 'contentRating', 'appId']

In [6]:
# Filtering dictionary so I only get results I want.

filteredresult = dict((k, result[k]) for k in featurekeys if k in result)
filteredresult

{'title': 'Pokémon GO',
 'installs': '100,000,000+',
 'minInstalls': 100000000,
 'score': 4.049132,
 'ratings': 13962045,
 'reviews': 5636673,
 'histogram': [2109097, 471206, 871610, 1682837, 8827295],
 'price': 0,
 'free': True,
 'offersIAP': True,
 'size': 'Varies with device',
 'genre': 'Adventure',
 'genreId': 'GAME_ADVENTURE',
 'released': 'Jul 6, 2016',
 'contentRating': 'Everyone',
 'appId': 'com.nianticlabs.pokemongo'}

In [7]:
# Now to make a function that gives me the filtered results.

def givemeinfo(appid):
    result = app(
    appid,
    lang='en',
    country='us')
    
    featurekeys = ['title', 'installs', 'minInstalls', 'score', 'ratings', 'reviews', 'histogram', 'price', 'free', 'offersIAP', 'size', 'genre', 'genreId', 'released', 'contentRating', 'appId']
    
    filteredresult = dict((k, result[k]) for k in featurekeys if k in result)
    
    return filteredresult

In [8]:
givemeinfo("com.gravity.romNAg")

{'title': 'Ragnarok M: Eternal Love(ROM)',
 'installs': '1,000,000+',
 'minInstalls': 1000000,
 'score': 3.8856142,
 'ratings': 76883,
 'reviews': 37853,
 'histogram': [14644, 3244, 4861, 7646, 46488],
 'price': 0,
 'free': True,
 'offersIAP': True,
 'size': '74M',
 'genre': 'Role Playing',
 'genreId': 'GAME_ROLE_PLAYING',
 'released': 'Jan 8, 2019',
 'contentRating': 'Teen',
 'appId': 'com.gravity.romNAg'}

### We see that there are 37853 reviews at the time of this scrape. [11/10/2020] at 1:47 PM EST.

==================================================================================================================

# Getting reviews for the game

In [9]:
# Now to get a list of all reviews for this game.

from google_play_scraper import Sort, reviews

In [10]:
# filter only the results that we want.

def givemereviews(appids):
    
    # Parts of the review that we are interested in
    keys = {'content', 'score', 'thumbsUpCount', 'reviewCreatedVersion'}
    
    # The function itself
    result, continuation_token = reviews(
    appids,
    lang='en',
    country='us',
    sort=Sort.RATING,
    count=10000,
)
    # Filtering the results
    filteredresults = [{k:v for k, v in i.items() if k in keys} for i in result]
    for item in filteredresults:
        item.update( {"appId":appids})
    return filteredresults

In [11]:
starttime = datetime.now()

reviewlist = givemereviews('com.gravity.romNAg')

print(datetime.now() - starttime)
print('------------')
print(reviewlist[:5])

0:00:12.867455
------------
[{'content': 'pretty good', 'score': 5, 'thumbsUpCount': 0, 'reviewCreatedVersion': '1.0.1', 'appId': 'com.gravity.romNAg'}, {'content': 'oh how I have missed this game', 'score': 5, 'thumbsUpCount': 0, 'reviewCreatedVersion': '1.0.1', 'appId': 'com.gravity.romNAg'}, {'content': "Just like the original Ragnarok Online on PC. This brings back a lot of memories. The gameplay is fantastic. It's an anime based open world MMO with lots of quests and things to complete. Socialize with other players, form parties, get married, etc. There are optional microtransactions and this game is very P2W in that you can use your $$$ to buy money/items. Don't let that stop you from having fun in this game as it is one of the few P2W games that is still 100% enjoyable as a F2P.", 'score': 5, 'thumbsUpCount': 1, 'reviewCreatedVersion': '1.0.5', 'appId': 'com.gravity.romNAg'}, {'content': 'Good so far', 'score': 5, 'thumbsUpCount': 0, 'reviewCreatedVersion': '1.2.2', 'appId': 'co

In [12]:
# Seems like we only have 6432 reviews to work with regardless of sorting method used.
# Only 6432 reviews are categorized as English from the US.

len(reviewlist)

6432

In [15]:
# We will use bz2 to compress pickle files to 1/4th the size.

import pickle
import bz2

# This is to compress it.
def compressed_pickle(title, data):
    with bz2.BZ2File(title + '.pbz2', 'w') as f:
        pickle.dump(data, f)
        
# This is to bring it back.        
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = pickle.load(data)
    return data

In [16]:
compressed_pickle('compressedlistreview', reviewlist)